# Scraper commands for publications on springer
## Examples

Here it is only about the steps, to parse are necessary so that it can be included in the end.

There are different website types for publications on springer.

In [1]:
journal_doi = 'https://doi.org/10.1007/s12525-020-00445-0'
journal_full_link = 'https://link.springer.com/article/10.1007/s12525-020-00445-0'

conference_chapter_link = 'https://link.springer.com/chapter/10.1007/978-3-030-49570-1_14'
conference_book_link = 'https://link.springer.com/book/10.1007/978-3-642-22531-4'

volume_contribution_link = 'https://link.springer.com/chapter/10.1007/978-3-030-06234-7_27'
#volume_link = 'https://link.springer.com/book/10.1007/978-3-030-06234-7'

springer_links = [journal_full_link, conference_chapter_link, conference_book_link, volume_contribution_link]

In [2]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
import json

In [3]:
# identify the type of the link
def get_springer_link_type(url):
    if '/chapter/' in url:
        return 'chapter'
    elif '/book/' in url:
        return 'book'
    elif '/article/' in url:
        return 'article'
    else:
        return 'unknown'


In [4]:
for link in springer_links:
    get_springer_link_type(link)

In [5]:
def get_bs(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15'}
        r = requests.get(url, headers=headers)
        print(r.status_code)
        bs = BeautifulSoup(r.text, 'html.parser')
    except:
        print('Error: ', url)
        return None
    return bs

In [6]:
journal_soup = get_bs(journal_full_link)
conference_chapter_soup = get_bs(conference_chapter_link)
conference_book_soup = get_bs(conference_book_link)
volume_contribution_soup = get_bs(volume_contribution_link)
springer_soups = [journal_soup, conference_chapter_soup, conference_book_soup, volume_contribution_soup]

200
200
200
200


In [7]:
# get whole data from json and loads to dict
# articles need to be prefiltered because they are nested differently
def get_json_data(bs):
    json_string = bs.find('script', {'type': 'application/ld+json'}).text
    json_data = json.loads(json_string)

    if '{"mainEntity":' in json_string:
        return json_data['mainEntity']
    else:
        return json_data


Creating json files for the three different types of publications for testing

In [8]:
journal_json = get_json_data(journal_soup)
conference_chapter_json = get_json_data(conference_chapter_soup)
conference_book_json = get_json_data(conference_book_soup)
volume_contribution_json = get_json_data(volume_contribution_soup)
json_data_list = [journal_json, conference_chapter_json, conference_book_json, volume_contribution_json]

In [9]:
# get names of json fields
def get_json_fields(json_data):
    return list(json_data.keys())

In [10]:
get_json_fields(journal_json)

['headline',
 'description',
 'datePublished',
 'dateModified',
 'pageStart',
 'pageEnd',
 'license',
 'sameAs',
 'keywords',
 'image',
 'isPartOf',
 'publisher',
 'author',
 'isAccessibleForFree',
 '@type']

## Main Fields
Title #TODO get title also for book (if instruction or with json)

In [11]:
def get_title(bs):
    try:
        title = bs.find('h1', {'class': 'c-article-title'}).text
        return title
    except:
        return None

In [12]:
for soup in springer_soups:
    print(get_title(soup))

Exploring customers’ likeliness to use e-service touchpoints in brick and mortar retail
A Two-Phase Framework for Detecting Manipulation Campaigns in Social Media
None
Applications of Artificial Intelligence in Supply Chain Management and Logistics: Focusing Onto Recognition for Supply Chain Execution


### Authors

In [13]:
def get_authors(bs):
    """
    Return list of authors in the format:
    [{'name': 'Author Name', 'orcid': orcid}, ...]

    :param bs: Received bs of the publication
    :return: list of dicts
    """
    try:
        json_data = get_json_data(bs)
        authors = []
        for author in json_data.get('author'):
            name = author.get('name')
            # split name at comma and reverse
            name = name.split(', ')
            name = name[1] + ' ' + name[0]
            orcid = author.get('url')
            authors.append({'name': name,
                            'orcid': orcid})
        return authors
    except:
        return None

In [14]:
for soup in springer_soups:
    print(get_authors(soup))

[{'name': 'Benjamin Barann', 'orcid': 'http://orcid.org/0000-0002-1965-2688'}, {'name': 'Jan H. Betzing', 'orcid': None}, {'name': 'Marco Niemann', 'orcid': None}, {'name': 'Benedikt Hoffmeister', 'orcid': None}, {'name': 'Jörg Becker', 'orcid': None}]
[{'name': 'Dennis Assenmacher', 'orcid': None}, {'name': 'Lena Clever', 'orcid': None}, {'name': 'Janina Susanne Pohl', 'orcid': None}, {'name': 'Heike Trautmann', 'orcid': None}, {'name': 'Christian Grimme', 'orcid': None}]
None
[{'name': 'Bernd Hellingrath', 'orcid': None}, {'name': 'Sandra Lechtenberg', 'orcid': None}]


### Keywords #TODO may be replaced with BS scraping because weird data structure

In [15]:
def get_Keywords(bs):
    """
    Return list of keywords from json data
    :param bs: Received bs of the publication
    :return:
    """

    """
    Return list of keywords in the format:
    [keyword1, keyword2, ...]
    :param bs: Received bs of the publication
    :return: list: String
    """
    try:
        json_data = get_json_data(bs)
        keywords_string = json_data.get('keywords')
        keywords = keywords_string.split(',')
        return keywords
    except:
        print("Error: no keywords found")
        return None

In [16]:
for soup in springer_soups:
    print(get_Keywords(soup))

['IT in Business', 'e-Commerce/e-business']
['Social campaign detection', ' Stream clustering', ' Unsupervised learning']
['Java', ' XQuery', ' abstract interpretation', ' higher-order patterns', ' non-deterministic functions']
['']


### Abstract

In [17]:
def get_abstract(bs, url):
    """
    Returns the abstract of the articles, books/preceedings do not have abstracts.
    :param json_data: Received json data of the publication
    :param bs: Received bs of the publication
    :return: String
    """
    if '/book/' in url:
        return None

    try:
        json_data = get_json_data(bs)
        abstract = json_data.get('description')
        return abstract
    except:
        print("Error: no abstract found")
        return None
    return None


In [18]:
for soup, url in zip(springer_soups, springer_links):
    print(get_abstract(soup, url))
    print("-------")

E-commerce has embraced the digital transformation and innovated with e-service touchpoints to improve customers’ experiences. Now some traditional, less-digitalized brick and mortar (BaM) retailers are starting to counteract the increasing competition by adopting digital touchpoints. However, the academic literature offers little in terms of what determines customers’ behavioral intentions toward e-service touchpoints. Therefore, drawing from the dominant design theory, this article first conceptually adapts selected dominant touchpoints of leading e-commerce solutions to BaM retail. Then 250 shoppers are surveyed regarding the likeliness that they will use the selected touchpoints, followed by an exploratory factor analysis to determine the touchpoints’ characteristics that lead to the shoppers’ assessments. The results suggest that customers prefer touchpoints that support product search and selection, provide information, and increase shopping efficiency. The likeliness that survey

### Pdf #todo #TODO: download

In [19]:
def get_pdf(bs, url):
    """
    Returns the pdf link of the publication, if available. Download might require login.
    :param bs: Received bs of the publication
    :param url:
    :return:
    """
    #todo automate download
    # differentiate between article, chapter and book
    if '/article/' in url:
        pdf = bs.find('div', class_='c-pdf-container').find('a', {'data-article-pdf': 'true'}).get('href')

    elif '/chapter/' in url:
        pdf_box = bs.find('div', {'class': 'c-article-access-provider'})
        pdf = pdf_box.find('a', {'data-track-action': 'Pdf download'}).get('href')

    elif '/book/' in url:
        pdf = bs.find('div', {'data-test': 'download-article-link-wrapper',
                              'class': 'js-context-bar-sticky-point-desktop'}).find('a', {
            'data-track-action': 'Book download - pdf'}).get('href')

    # append base url if necesary
    if 'link.springer.com' in pdf:
        return pdf
    else:
        return f'https://link.springer.com{pdf}'

In [20]:
test_links = []
for soup, url in zip(springer_soups, springer_links):
    test_links.append(get_pdf(soup, url))

test_links

['https://link.springer.com/content/pdf/10.1007/s12525-020-00445-0.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-030-49570-1_14.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-642-22531-4.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-030-06234-7_27.pdf']

In [21]:
journal_soup.find('div', class_='c-pdf-container').find('a', {'data-article-pdf': 'true'}).get('href')

'https://link.springer.com/content/pdf/10.1007/s12525-020-00445-0.pdf'

### Publisher

In [22]:
def get_publisher(bs):
    """
    Returns the publisher of the publication
    :param bs: Received bs of the publication
    :return: String
    """
    try:
        json_data = get_json_data(bs)
        publisher = json_data.get('publisher').get('name')
        return publisher
    except:
        print("Error: no publisher found")
        return None

In [23]:
for soup in springer_soups:
    print(get_publisher(soup))

Springer Berlin Heidelberg
Springer International Publishing
Springer Berlin Heidelberg
Springer International Publishing


### Year

In [24]:
def get_year(bs, url):
    """
    Returns the year of the publication
    :param bs: Received bs of the publication
    :return:
    """
    try:
        json_data = get_json_data(bs)
        if ('/chapter/' in url) or ('/article/') in url:
            date = json_data.get('datePublished')
            year = date.split('-')[0]  # get year from date (if date is available)
            return year
        if '/book/' in url:
            year = json_data.get('copyrightYear')
            return year

    except:
        print("Error: no year found")
        print(bs.name)
        return None

In [27]:
for soup, url in zip(springer_soups, springer_links):
    print(get_year(soup, url))


2020
2020
2011
2019


### Publication type

In [38]:
def get_publication_type(bs, url):
    """
    Returns the publication type of the publication
    :param bs: Received bs of the publication
    :return: String
    """
    try:
        if '/book/' in url:
            type = bs.find('li', {'class': 'c-article-identifiers__item'}).text
        else:
            type = bs.find('li', {'class': 'c-article-identifiers__item', 'data-test': 'article-category'}).text
        return type
    except:
        print("Error: no publication type found in bs, deriving by url")
        if '/book/' in url:
            return 'book'
        elif '/chapter/' in url:
            return 'chapter'
        elif '/article/' in url:
            return 'article'
        #return None

In [39]:
for soup, url in zip(springer_soups, springer_links):
    print(get_publication_type(soup, url))

Research Paper
Conference paper
Conference proceedings
Chapter
